# Adjacency Matrix (AM) as HRT (Hash Relational Tensor)

>hash-addressable, dynamically-growing, PyTorch-only implementation

```math
r(hᵢ, hⱼ) ← v  (\text{ overwrite if exists, insert if new})
```

for an arbitrary number of tokens whose only identifier is the hash value h = hash(tᵢ).


No Python loops touch the tensor itself; all lookups and updates are O(1) and executed with one tensor op.

## Data layout

We keep a single 2-D tensor R of shape (capacity, capacity)
row/col indices are internal integer ids (0,1,2,…) that we map to/from the hash values with two dictionaries:

```text
hash → idx   : self.h2i
idx  → hash  : self.i2h
```

capacity grows automatically in powers of two (doubling when >75 % full) and the old contents are copied into the new larger tensor with a single index_copy.

In [1]:
import torch
import os

# Check available GPUs
print("Available GPUs:")
for i in range(torch.cuda.device_count()):
    props = torch.cuda.get_device_properties(i)
    print(f"  GPU {i}: {props.name}")
    print(f"    Compute Capability: sm_{props.major}{props.minor}")
    print(f"    Memory: {props.total_memory / 1024**3:.2f} GB")

# Select RTX 3060 (adjust index based on output above)
# Option A: Hide Quadro, only show RTX
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Assuming RTX is at index 1
DEVICE = torch.device("cuda:0")  # Now index 0 refers to the RTX

# Option B: Explicitly select by index (if you don't set CUDA_VISIBLE_DEVICES)
# DEVICE = torch.device("cuda:1")  # Direct access to RTX at original index 1

print(f"\nUsing device: {DEVICE}")
if DEVICE.type == "cuda":
    print(f"Device name: {torch.cuda.get_device_name(DEVICE)}")
    print(f"Memory: {torch.cuda.get_device_properties(DEVICE).total_memory / 1024**3:.2f} GB")

Available GPUs:
  GPU 0: NVIDIA GeForce RTX 3060
    Compute Capability: sm_86
    Memory: 11.66 GB
  GPU 1: Quadro M1200
    Compute Capability: sm_50
    Memory: 3.94 GB

Using device: cuda:0
Device name: NVIDIA GeForce RTX 3060
Memory: 11.66 GB


/home/alexmy/SGS/SGS_lib/hllset-swarm-kimi/.venv/lib64/python3.12/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU1 Quadro M1200 which is of cuda capability 5.0.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (7.0) - (12.0)
    
  warnings.warn(
/home/alexmy/SGS/SGS_lib/hllset-swarm-kimi/.venv/lib64/python3.12/site-packages/torch/cuda/__init__.py:304: UserWarning: 
    Please install PyTorch with a following CUDA
    configurations:  12.6 following instructions at
    https://pytorch.org/get-started/locally/
    
  warnings.warn(matched_cuda_warn.format(matched_arches))
/home/alexmy/SGS/SGS_lib/hllset-swarm-kimi/.venv/lib64/python3.12/site-packages/torch/cuda/__init__.py:326: UserWarning: 
Quadro M1200 with CUDA capability sm_50 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_70 sm_75 sm_80 sm_86 sm_90 sm_100 sm_120.
If you want to use the Quadro M1200 GPU with P

In [2]:
from src.hllset_swarm.hrt import HRT

## Usage demo

In [3]:
dev = 'cuda' if torch.cuda.is_available() else 'cpu'
rel = HRT(device=dev)

tok_hashes = [hash(t) for t in ["cat", "dog", "bird"]]

rel.update(tok_hashes[0], tok_hashes[1], 0.8)   # r(cat,dog)=0.8
rel.update(tok_hashes[1], tok_hashes[2], 0.5)   # r(dog,bird)=0.5
rel.update(tok_hashes[0], tok_hashes[1], 0.9)   # overwrite with 0.9

print(rel.get(tok_hashes[0], tok_hashes[1]))    # tensor(0.9, device='cuda:0')
print(rel.get_dense())                          # 3×3 dense sub-matrix

tensor(0.9000, device='cuda:0')
tensor([[0.0000, 0.9000, 0.0000],
        [0.0000, 0.0000, 0.5000],
        [0.0000, 0.0000, 0.0000]], device='cuda:0')


In [4]:
rel = HRT()

# build a tiny time graph
t = [hash(w) for w in ["t0", "t1", "t2", "t3"]]
rel.update(t[0], t[1], 0.9)   # t0 → t1
rel.update(t[1], t[2], 0.8)   # t1 → t2
rel.update(t[2], t[3], 0.7)   # t2 → t3
rel.update(t[0], t[3], 0.5)   # t0 → t3  (skip connection)

print(rel.get_history(t[3], horizon=2))
# {hash(t2): 0.7, hash(t0): 0.5}   two strongest predecessors

print(rel.get_future(t[0], horizon=2))
# {hash(t1): 0.9, hash(t3): 0.5}   two strongest successors

print(rel.project([t[0], t[2], t[3]]))
# 3×3 dense tensor of relations among exactly these three tokens

{7604630207966975244: 0.699999988079071, 6685043807368327666: 0.5}
{-4822480586015876194: 0.8999999761581421, -6830413446406971881: 0.5}
tensor([[0.0000, 0.0000, 0.5000],
        [0.0000, 0.0000, 0.7000],
        [0.0000, 0.0000, 0.0000]])


## Pruning Demo

### Complexity

Time: O(|keep|²) – once.
Memory: the old matrix is discarded immediately; only the pruned one remains.
All subsequent operations (update, get_history, project, …) run on the smaller matrix.

In [5]:
rel = HRT()

# build a graph with 4 tokens
t = [hash(w) for w in ["A", "B", "C", "D"]]
rel.update(t[0], t[1], 1.0)
rel.update(t[1], t[2], 2.0)
rel.update(t[2], t[3], 3.0)
rel.update(t[0], t[3], 4.0)

print("before prune", rel.R.shape)      # 4×4

pruned = rel.prune([t[0], t[2], t[3]])  # drop B
print("after prune", pruned.shape)      # 3×3
print(rel.h2i)                          # {hash_A:0, hash_C:1, hash_D:2}

# history/future now work only on the pruned set
print(rel.get_history(t[3]))            # {hash_A:4.0, hash_C:3.0}

before prune torch.Size([16, 16])
after prune torch.Size([3, 3])
{-9176445229405775260: 0, 7944667499784224323: 1, -9135990848321305430: 2}
{-9176445229405775260: 4.0}


## Complexity

- update/get: O(1) average time, one tensor write.
- memory: O(N²) where N = number of distinct hashes (only the actually used N×N block is returned by get_dense()).
- growth: amortised O(1); when capacity doubles we pay one index_copy of the existing N×N block.

## Extensions

1. Symmetric relation: mirror each write R[j,i] = R[i,j].
2. Directed vs undirected: store only upper triangle and use a triu_indices view.
3. Mini-batch updates: vectorise with index_put_ and two integer tensors rows, cols.